In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import os
#import ipdb

import matplotlib as mpl
from matplotlib import lines
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [8]:
EXP_NAME = "std_dev"
# Read all standard deviation files of each gateway node and count what category nodes are at each time period
def read_std():
    INPUT_DIR = os.path.join( os.getcwd(), EXP_NAME)
    data = pd.DataFrame()
    dirs = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR))]    
    for client in dirs:
        client_data = pd.read_csv(os.path.join(INPUT_DIR, client), sep=',')
        client_data = client_data.set_index(pd.DatetimeIndex(client_data.time))
        count_good = 0
        count_bad = 0
        count_inconsistent = 0
        #client_data = client_data.tail(1000)
        #['general_std','general_mean','group_std','group_mean','latency']
        client_data = client_data.ix['2019-01-24 00:00':'2019-01-25 00:00']
        #client_data['latency'].plot()
        #plt.ylim(0,3)
        #plt.show()
        print(client, len(client_data))
        for index, row in client_data.iterrows():                   
            client_data.loc[index, 'gateway'] = client
            if row['general_mean'] is None:
                continue
            if row['latency'] <= row['general_mean']:
                if row['group_std'] <= row['general_std']:
                    client_data.loc[index, 'category'] = 'Good'
                    count_good +=1
                else:
                    if count_inconsistent<count_good:
                        client_data.loc[index, 'category'] = 'Inconsistent good'
                    else:
                        client_data.loc[index, 'category'] = 'Inconsistent'
                    count_inconsistent +=1
            else:
                if row['group_std'] < row['general_std']:                    
                    if count_inconsistent<count_good:
                        client_data.loc[index, 'category'] = 'Inconsistent good'
                    else:
                        client_data.loc[index, 'category'] = 'Inconsistent'
                    count_inconsistent +=1
                else:
                    client_data.loc[index, 'category'] = 'Bad'
                    count_bad +=1
            client_data.loc[index, 'count_good'] = count_good
            client_data.loc[index, 'count_bad'] = count_bad
            client_data.loc[index, 'count_inconsistent'] = count_inconsistent
        
        data = pd.concat([data, client_data])
    return data

In [9]:
dataset = read_std()

print(dataset)

/home/khulan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


proxy.malla.guifi.net 1428
proxy.muntanyola.guifi.net 1428
proxy.olost.guifi.net 1376
10.228.193.210 0
10.228.192.210 1441
10.139.94.98 1429
10.139.37.194 1441
proxy.sbll.guifi.net 1375
10.138.57.2 1441
10.138.25.67 1441
10.139.17.4 1441
10.138.85.130 1441
10.138.120.66 1441
proxy.perafita.guifi.net 1376
10.139.40.122 1441
proxy.elpipa.guifi.net 1429
10.140.93.35 1440
proxy.smc.guifi.net 603
10.139.40.85 1441
proxy.cecilia.guifi.net 1429
                         category  count_bad  count_good  count_inconsistent  \
time                                                                           
2019-01-24 00:00:00  Inconsistent        0.0         0.0                 1.0   
2019-01-24 00:01:00  Inconsistent        0.0         0.0                 2.0   
2019-01-24 00:02:00          Good        0.0         1.0                 2.0   
2019-01-24 00:03:00  Inconsistent        0.0         1.0                 3.0   
2019-01-24 00:04:00           Bad        1.0         1.0                 3.0  